## Solution for Lab 3

Data Mining course <br>
Student: Danis Alukaev <br>
Email: d.alukaev@innopolis.university <br> 
Group: B19-DS-01 <br>

### 0. Prerequisites

In [1]:
import pandas as pd
import numpy as np
import itertools

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import OneHotEncoder
from catboost import CatBoostClassifier
from sklearn.metrics import balanced_accuracy_score

import warnings
warnings.filterwarnings("ignore")

### 1. Data Preparation

In [2]:
def load_data(sample=None):
    df = pd.read_csv("./data/completed_orders.csv")
    if sample:
        size = int(len(df) * sample)
        df = df.sample(n=size)
    return df

In [3]:
def get_map():
    data = pd.read_csv("./data/completed_orders.csv")
    df = data.copy()[["StockCode", "Description"]]
    df.drop_duplicates(inplace=True)
    df = df.set_index("StockCode")
    return df

In [4]:
def get_all_combinations(df, sample=None):
    all_stockcodes, all_customers = set(df["StockCode"].to_numpy()), set(df["CustomerID"].to_numpy())
    all_combinations = np.array(list(itertools.product(all_customers, all_stockcodes)))
    all_combinations = pd.DataFrame(data=all_combinations, columns=["CustomerID", "StockCode"])
    if sample:
        size = int(len(all_combinations) * sample)
        all_combinations = all_combinations.sample(n=size)
    return all_combinations

In [5]:
def split_data(df, all_combinations):
    train, test = train_test_split(df, test_size=0.2, shuffle=False)
    train["Reorder"], test["Reorder"] = 1, 1
    train = pd.merge(train, all_combinations, how="outer", on=["CustomerID", "StockCode"]).fillna(0)
    test = pd.merge(test, all_combinations, how="outer", on=["CustomerID", "StockCode"]).fillna(0)
    train.drop_duplicates(inplace=True)
    test.drop_duplicates(inplace=True)
    train = train.sample(frac=1.0)
    test = test.sample(frac=1.0)
    return train, test

In [6]:
df = load_data(sample=0.1)
data = df.copy()[["CustomerID", "StockCode"]]
all_combinations = get_all_combinations(data, sample=0.05)
train, test = split_data(data, all_combinations)

### 2. PCA

In [7]:
def encode_data(df):
    X, y = df[["CustomerID", "StockCode"]], df["Reorder"]
    all_stockcodes, all_customers = set(df["StockCode"].to_numpy()), set(df["CustomerID"].to_numpy())
    
    encoder_customer = OneHotEncoder(handle_unknown='ignore', sparse=False)
    encoder_stockcodes = OneHotEncoder(handle_unknown='ignore', sparse=False)
    
    encoder_customer.fit(np.array(list(all_customers))[..., np.newaxis])
    encoder_stockcodes.fit(np.array(list(all_stockcodes))[..., np.newaxis])
    
    customer_encoded = pd.DataFrame(encoder_customer.transform(X["CustomerID"][..., np.newaxis]))
    customer_encoded.rename(columns={i: f"c{i}" for i in range(len(encoder_customer.categories_[0]))}, inplace=True)
    
    stockcodes_encoded = pd.DataFrame(encoder_stockcodes.transform(X["StockCode"][..., np.newaxis]))
    stockcodes_encoded.rename(columns={i: f"s{i}" for i in range(len(encoder_stockcodes.categories_[0]))}, inplace=True)
    
    X = pd.concat([customer_encoded, stockcodes_encoded], axis=1)
    
    del stockcodes_encoded
    del customer_encoded
    
    return X, y

In [8]:
X_train, y_train = encode_data(train)

In [9]:
X_test, y_test = encode_data(test)

In [10]:
pca = PCA(n_components=6)

X_train = pca.fit_transform(X_train)
X_test = pca.fit_transform(X_test)

In [11]:
pca.explained_variance_

array([0.00260294, 0.00245297, 0.00224153, 0.00222006, 0.0021025 ,
       0.00207038])

### 3. Building Model

In [12]:
model = CatBoostClassifier(iterations=7000,
                           task_type="GPU",
                           learning_rate=0.03,
                           depth=5,
                           devices='0',
                           eval_metric='AUC')

In [13]:
model.fit(X_train,
          y_train,
          verbose=True)

0:	learn: 0.7276220	total: 5.95ms	remaining: 41.6s
1:	learn: 0.7467943	total: 20.2ms	remaining: 1m 10s
2:	learn: 0.7486185	total: 31.7ms	remaining: 1m 13s
3:	learn: 0.7544108	total: 43.1ms	remaining: 1m 15s
4:	learn: 0.7583447	total: 54.8ms	remaining: 1m 16s
5:	learn: 0.7594923	total: 66.3ms	remaining: 1m 17s
6:	learn: 0.7614708	total: 77.9ms	remaining: 1m 17s
7:	learn: 0.7629622	total: 89.5ms	remaining: 1m 18s
8:	learn: 0.7645338	total: 101ms	remaining: 1m 18s
9:	learn: 0.7666999	total: 113ms	remaining: 1m 18s
10:	learn: 0.7686384	total: 124ms	remaining: 1m 18s
11:	learn: 0.7691658	total: 136ms	remaining: 1m 19s
12:	learn: 0.7696136	total: 142ms	remaining: 1m 16s
13:	learn: 0.7700039	total: 147ms	remaining: 1m 13s
14:	learn: 0.7699579	total: 153ms	remaining: 1m 11s
15:	learn: 0.7701491	total: 159ms	remaining: 1m 9s
16:	learn: 0.7701336	total: 164ms	remaining: 1m 7s
17:	learn: 0.7702338	total: 170ms	remaining: 1m 5s
18:	learn: 0.7731380	total: 176ms	remaining: 1m 4s
19:	learn: 0.774224

171:	learn: 0.7840634	total: 1s	remaining: 39.8s
172:	learn: 0.7841084	total: 1.01s	remaining: 39.8s
173:	learn: 0.7841590	total: 1.01s	remaining: 39.7s
174:	learn: 0.7841971	total: 1.02s	remaining: 39.7s
175:	learn: 0.7842298	total: 1.02s	remaining: 39.7s
176:	learn: 0.7842507	total: 1.03s	remaining: 39.7s
177:	learn: 0.7842638	total: 1.03s	remaining: 39.6s
178:	learn: 0.7842943	total: 1.04s	remaining: 39.6s
179:	learn: 0.7843201	total: 1.04s	remaining: 39.6s
180:	learn: 0.7843648	total: 1.05s	remaining: 39.6s
181:	learn: 0.7843896	total: 1.05s	remaining: 39.6s
182:	learn: 0.7844015	total: 1.06s	remaining: 39.5s
183:	learn: 0.7844673	total: 1.07s	remaining: 39.5s
184:	learn: 0.7844938	total: 1.07s	remaining: 39.5s
185:	learn: 0.7845224	total: 1.08s	remaining: 39.5s
186:	learn: 0.7845676	total: 1.08s	remaining: 39.4s
187:	learn: 0.7845930	total: 1.09s	remaining: 39.4s
188:	learn: 0.7846076	total: 1.09s	remaining: 39.4s
189:	learn: 0.7846370	total: 1.1s	remaining: 39.4s
190:	learn: 0.78

359:	learn: 0.7882668	total: 2s	remaining: 36.8s
360:	learn: 0.7882818	total: 2s	remaining: 36.8s
361:	learn: 0.7882886	total: 2.01s	remaining: 36.8s
362:	learn: 0.7883055	total: 2.01s	remaining: 36.8s
363:	learn: 0.7883388	total: 2.02s	remaining: 36.8s
364:	learn: 0.7883545	total: 2.02s	remaining: 36.8s
365:	learn: 0.7883636	total: 2.03s	remaining: 36.8s
366:	learn: 0.7883807	total: 2.03s	remaining: 36.8s
367:	learn: 0.7884227	total: 2.04s	remaining: 36.7s
368:	learn: 0.7884538	total: 2.04s	remaining: 36.7s
369:	learn: 0.7884717	total: 2.05s	remaining: 36.7s
370:	learn: 0.7884762	total: 2.05s	remaining: 36.7s
371:	learn: 0.7885077	total: 2.06s	remaining: 36.7s
372:	learn: 0.7885322	total: 2.06s	remaining: 36.7s
373:	learn: 0.7885635	total: 2.07s	remaining: 36.7s
374:	learn: 0.7885796	total: 2.08s	remaining: 36.7s
375:	learn: 0.7886017	total: 2.08s	remaining: 36.6s
376:	learn: 0.7886190	total: 2.08s	remaining: 36.6s
377:	learn: 0.7886446	total: 2.09s	remaining: 36.6s
378:	learn: 0.7886

545:	learn: 0.7912189	total: 3s	remaining: 35.4s
546:	learn: 0.7912338	total: 3s	remaining: 35.4s
547:	learn: 0.7912398	total: 3.01s	remaining: 35.4s
548:	learn: 0.7912515	total: 3.01s	remaining: 35.4s
549:	learn: 0.7912681	total: 3.02s	remaining: 35.4s
550:	learn: 0.7912805	total: 3.02s	remaining: 35.4s
551:	learn: 0.7912954	total: 3.02s	remaining: 35.3s
552:	learn: 0.7913031	total: 3.03s	remaining: 35.3s
553:	learn: 0.7913178	total: 3.04s	remaining: 35.3s
554:	learn: 0.7913292	total: 3.04s	remaining: 35.3s
555:	learn: 0.7913373	total: 3.04s	remaining: 35.3s
556:	learn: 0.7913463	total: 3.05s	remaining: 35.3s
557:	learn: 0.7913653	total: 3.06s	remaining: 35.3s
558:	learn: 0.7913852	total: 3.06s	remaining: 35.3s
559:	learn: 0.7913992	total: 3.07s	remaining: 35.3s
560:	learn: 0.7914070	total: 3.07s	remaining: 35.3s
561:	learn: 0.7914228	total: 3.08s	remaining: 35.2s
562:	learn: 0.7914351	total: 3.08s	remaining: 35.2s
563:	learn: 0.7914415	total: 3.08s	remaining: 35.2s
564:	learn: 0.7914

716:	learn: 0.7934572	total: 3.78s	remaining: 33.2s
717:	learn: 0.7934622	total: 3.79s	remaining: 33.1s
718:	learn: 0.7934781	total: 3.79s	remaining: 33.1s
719:	learn: 0.7934887	total: 3.8s	remaining: 33.1s
720:	learn: 0.7935070	total: 3.8s	remaining: 33.1s
721:	learn: 0.7935126	total: 3.81s	remaining: 33.1s
722:	learn: 0.7935236	total: 3.81s	remaining: 33.1s
723:	learn: 0.7935337	total: 3.81s	remaining: 33.1s
724:	learn: 0.7935394	total: 3.82s	remaining: 33.1s
725:	learn: 0.7935578	total: 3.82s	remaining: 33s
726:	learn: 0.7935667	total: 3.83s	remaining: 33s
727:	learn: 0.7935703	total: 3.83s	remaining: 33s
728:	learn: 0.7935831	total: 3.84s	remaining: 33s
729:	learn: 0.7935958	total: 3.84s	remaining: 33s
730:	learn: 0.7936093	total: 3.85s	remaining: 33s
731:	learn: 0.7936124	total: 3.85s	remaining: 33s
732:	learn: 0.7936197	total: 3.86s	remaining: 33s
733:	learn: 0.7936421	total: 3.86s	remaining: 33s
734:	learn: 0.7936534	total: 3.87s	remaining: 32.9s
735:	learn: 0.7936699	total: 3.8

...

6210:	learn: 0.8145505	total: 30.3s	remaining: 3.84s
6211:	learn: 0.8145529	total: 30.3s	remaining: 3.84s
6212:	learn: 0.8145546	total: 30.3s	remaining: 3.83s
6213:	learn: 0.8145552	total: 30.3s	remaining: 3.83s
6214:	learn: 0.8145556	total: 30.3s	remaining: 3.82s
6215:	learn: 0.8145580	total: 30.3s	remaining: 3.82s
6216:	learn: 0.8145599	total: 30.3s	remaining: 3.81s
6217:	learn: 0.8145609	total: 30.3s	remaining: 3.81s
6218:	learn: 0.8145628	total: 30.3s	remaining: 3.8s
6219:	learn: 0.8145651	total: 30.3s	remaining: 3.8s
6220:	learn: 0.8145652	total: 30.3s	remaining: 3.79s
6221:	learn: 0.8145658	total: 30.3s	remaining: 3.79s
6222:	learn: 0.8145654	total: 30.3s	remaining: 3.78s
6223:	learn: 0.8145681	total: 30.3s	remaining: 3.78s
6224:	learn: 0.8145694	total: 30.3s	remaining: 3.77s
6225:	learn: 0.8145707	total: 30.3s	remaining: 3.77s
6226:	learn: 0.8145708	total: 30.3s	remaining: 3.76s
6227:	learn: 0.8145751	total: 30.3s	remaining: 3.76s
6228:	learn: 0.8145761	total: 30.3s	remaining: 3

6378:	learn: 0.8147898	total: 31s	remaining: 3.02s
6379:	learn: 0.8147944	total: 31s	remaining: 3.02s
6380:	learn: 0.8147985	total: 31.1s	remaining: 3.01s
6381:	learn: 0.8147998	total: 31.1s	remaining: 3.01s
6382:	learn: 0.8148001	total: 31.1s	remaining: 3s
6383:	learn: 0.8148006	total: 31.1s	remaining: 3s
6384:	learn: 0.8148041	total: 31.1s	remaining: 2.99s
6385:	learn: 0.8148036	total: 31.1s	remaining: 2.99s
6386:	learn: 0.8148037	total: 31.1s	remaining: 2.98s
6387:	learn: 0.8148048	total: 31.1s	remaining: 2.98s
6388:	learn: 0.8148052	total: 31.1s	remaining: 2.97s
6389:	learn: 0.8148058	total: 31.1s	remaining: 2.97s
6390:	learn: 0.8148066	total: 31.1s	remaining: 2.96s
6391:	learn: 0.8148063	total: 31.1s	remaining: 2.96s
6392:	learn: 0.8148074	total: 31.1s	remaining: 2.95s
6393:	learn: 0.8148078	total: 31.1s	remaining: 2.95s
6394:	learn: 0.8148091	total: 31.1s	remaining: 2.94s
6395:	learn: 0.8148111	total: 31.1s	remaining: 2.94s
6396:	learn: 0.8148114	total: 31.2s	remaining: 2.94s
639

6552:	learn: 0.8150644	total: 32s	remaining: 2.19s
6553:	learn: 0.8150645	total: 32.1s	remaining: 2.18s
6554:	learn: 0.8150654	total: 32.1s	remaining: 2.18s
6555:	learn: 0.8150669	total: 32.1s	remaining: 2.17s
6556:	learn: 0.8150669	total: 32.1s	remaining: 2.17s
6557:	learn: 0.8150688	total: 32.1s	remaining: 2.16s
6558:	learn: 0.8150688	total: 32.1s	remaining: 2.16s
6559:	learn: 0.8150749	total: 32.1s	remaining: 2.15s
6560:	learn: 0.8150785	total: 32.1s	remaining: 2.15s
6561:	learn: 0.8150799	total: 32.1s	remaining: 2.14s
6562:	learn: 0.8150802	total: 32.1s	remaining: 2.14s
6563:	learn: 0.8150802	total: 32.1s	remaining: 2.13s
6564:	learn: 0.8150809	total: 32.1s	remaining: 2.13s
6565:	learn: 0.8150836	total: 32.1s	remaining: 2.12s
6566:	learn: 0.8150840	total: 32.1s	remaining: 2.12s
6567:	learn: 0.8150845	total: 32.1s	remaining: 2.11s
6568:	learn: 0.8150882	total: 32.1s	remaining: 2.11s
6569:	learn: 0.8150891	total: 32.1s	remaining: 2.1s
6570:	learn: 0.8150899	total: 32.1s	remaining: 2.

6715:	learn: 0.8152812	total: 32.8s	remaining: 1.39s
6716:	learn: 0.8152823	total: 32.8s	remaining: 1.38s
6717:	learn: 0.8152845	total: 32.9s	remaining: 1.38s
6718:	learn: 0.8152854	total: 32.9s	remaining: 1.37s
6719:	learn: 0.8152853	total: 32.9s	remaining: 1.37s
6720:	learn: 0.8152870	total: 32.9s	remaining: 1.36s
6721:	learn: 0.8152876	total: 32.9s	remaining: 1.36s
6722:	learn: 0.8152880	total: 32.9s	remaining: 1.35s
6723:	learn: 0.8152892	total: 32.9s	remaining: 1.35s
6724:	learn: 0.8152906	total: 32.9s	remaining: 1.34s
6725:	learn: 0.8152914	total: 32.9s	remaining: 1.34s
6726:	learn: 0.8152938	total: 32.9s	remaining: 1.33s
6727:	learn: 0.8152994	total: 32.9s	remaining: 1.33s
6728:	learn: 0.8153034	total: 32.9s	remaining: 1.32s
6729:	learn: 0.8153057	total: 32.9s	remaining: 1.32s
6730:	learn: 0.8153058	total: 32.9s	remaining: 1.31s
6731:	learn: 0.8153085	total: 32.9s	remaining: 1.31s
6732:	learn: 0.8153092	total: 32.9s	remaining: 1.3s
6733:	learn: 0.8153096	total: 32.9s	remaining: 

6883:	learn: 0.8155101	total: 33.6s	remaining: 567ms
6884:	learn: 0.8155105	total: 33.6s	remaining: 562ms
6885:	learn: 0.8155122	total: 33.6s	remaining: 557ms
6886:	learn: 0.8155189	total: 33.7s	remaining: 552ms
6887:	learn: 0.8155198	total: 33.7s	remaining: 547ms
6888:	learn: 0.8155191	total: 33.7s	remaining: 542ms
6889:	learn: 0.8155195	total: 33.7s	remaining: 538ms
6890:	learn: 0.8155196	total: 33.7s	remaining: 533ms
6891:	learn: 0.8155211	total: 33.7s	remaining: 528ms
6892:	learn: 0.8155230	total: 33.7s	remaining: 523ms
6893:	learn: 0.8155235	total: 33.7s	remaining: 518ms
6894:	learn: 0.8155239	total: 33.7s	remaining: 513ms
6895:	learn: 0.8155234	total: 33.7s	remaining: 508ms
6896:	learn: 0.8155281	total: 33.7s	remaining: 503ms
6897:	learn: 0.8155361	total: 33.7s	remaining: 498ms
6898:	learn: 0.8155361	total: 33.7s	remaining: 494ms
6899:	learn: 0.8155378	total: 33.7s	remaining: 489ms
6900:	learn: 0.8155380	total: 33.7s	remaining: 484ms
6901:	learn: 0.8155392	total: 33.7s	remaining:

In [14]:
y_pred = model.predict(X_test)

In [15]:
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
result = f"Balanced accuracy: {balanced_accuracy:.2f}"
print(result)

Balanced accuracy: 0.86


### 4. Inference

In [16]:
def predict_cart(customer_id):
    all_stockcodes, all_customers = set(df["StockCode"].to_numpy()), set(df["CustomerID"].to_numpy())
    X = np.array(list(itertools.product([customer_id], all_stockcodes)))
    X = pd.DataFrame(data=X, columns=["CustomerID", "StockCode"])
    
    encoder_customer = OneHotEncoder(handle_unknown='ignore', sparse=False)
    encoder_stockcodes = OneHotEncoder(handle_unknown='ignore', sparse=False)
    
    encoder_customer.fit(np.array(list(all_customers))[..., np.newaxis])
    encoder_stockcodes.fit(np.array(list(all_stockcodes))[..., np.newaxis])
    
    customer_encoded = pd.DataFrame(encoder_customer.transform(X["CustomerID"][..., np.newaxis]))
    customer_encoded.rename(columns={i: f"c{i}" for i in range(len(encoder_customer.categories_[0]))}, inplace=True)
    
    stockcodes_encoded = pd.DataFrame(encoder_stockcodes.transform(X["StockCode"][..., np.newaxis]))
    stockcodes_encoded.rename(columns={i: f"s{i}" for i in range(len(encoder_stockcodes.categories_[0]))}, inplace=True)
    
    X = pd.concat([customer_encoded, stockcodes_encoded], axis=1)
    
    cart_ = model.predict(X)
    
    mapping = get_map()
    
    indices = np.nonzero(cart_)[0]
    
    cart = []
    for idx in indices:
        index = encoder_stockcodes.categories_[0][idx]
        product = mapping.iloc[mapping.index.get_loc(index)]
        if len(product) > 1:
            product = product.iloc[0]

        cart.append(product.Description)
    
    return cart 

In [17]:
predict_cart(15809)

['vintage bead pink evening bag',
 'blue daisy mobile',
 'charlie & lola wastepaper bin flora',
 '6 rocket balloons ',
 'party pizza dish red retrospot',
 'food cover with beads set 2 ']